<a href="https://colab.research.google.com/github/kennynlam/DATASCI-112-Final-Project/blob/main/DATASCI%20112%20Final%20Project%20(Data%20Collection).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATASCI 112 Final Project - Data Collection**

For my final project, I decided to collect data to predict enrollment changes within Iowa public school districts. In Iowa, public school districts receive yearly funding based on the number of students enrolled at the start of a given school year. Essentially, if a school is able to predict whether their enrollment numbers will increase/decrease, they can then predict whether they will gain/lose funding & subsequently take proper action.

My proposal was that yearly changes in enrollment are somewhat predictable based on a given number of variables in a district's current year & from past trends. To potentially minimize the effect of the 2020-2021 pandemic school year, I decided to also compile enrollment data from 2019-2020 & county population data from 2019 as additional variables to potentially minimizes This notebook gathers data from a multitude of sources, cleans and formats each file, and then merges them all together in a cohesive dataframe.

Admittedly, the training data could be larger as it is centralized around the 2021-2022 school year alone. However, I ran into a variety of issues (impact of COVID-19, change in standardized testing in 2019, inconsistent formatting of data for certain datasets across different years) that ultimately prevented me from compiling a multi-year dataset. Instead, I decided to collect data on a larger number of metrics for the 2021-2022 school year, which I then could test to find an ideal combination that would yield the best predictions for the following year.

In this notebook, I also collected data on recently published 2022-2023 district enrollments as well as the state's official enrollment projection for that year. A goal of my model is hence to generate predictions that are more accurate than the state's projections. Since my model is limited to predicting the % change in enrollment from 2021-2022 to 2022-2023, I do plan on fitting 2020-2021 data as an external testing set to predict its respective % change in 2021-2022 & determining if there is a noticeable change in the model's accuracy.

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_dir = "/content/gdrive/My Drive/Colab Notebooks/DATASCI 112/Final Project/Data Files/"
# most files were originally excel spreadsheets available through the Iowa Department of Education's database & thus did not correctly export as CSV files
# much time was spent manualling fixing and adjusting each file in order to merge them all at the end
# county populations were obtained from an official government json api

# 1st merge: compiling general data on district enrollment from the past 2 previous years, current year (2021-22), and next year (what we are trying to predict!)
df_ppy_enrollment = pd.read_csv(data_dir + "2019-20 K12 ENROLLMENT.csv")
df_py_enrollment = pd.read_csv(data_dir + "2020-21 K12 ENROLLMENT.csv")
df_cy_enrollment = pd.read_csv(data_dir + "2021-22 K12 ENROLLMENT.csv")
df_ny_enrollment = pd.read_csv(data_dir + "2022-23 K12 ENROLLMENT.csv")
df_enrollment_proj = pd.read_csv(data_dir + "STATE ENROLLMENT PROJECTIONS1.csv")

import requests
response = requests.get("https://data.iowa.gov/resource/qtnr-zsrc.json?$query=SELECT%0A%20%20%60fips%60%2C%0A%20%20%60geographicname%60%2C%0A%20%20%60year%60%2C%0A%20%20%60population%60%2C%0A%20%20%60primary_point%60%2C%0A%20%20%60%3A%40computed_region_hhz5_dst4%60%2C%0A%20%20%60%3A%40computed_region_y683_txed%60%2C%0A%20%20%60%3A%40computed_region_g8ff_h7ce%60%0AWHERE%0A%20%20%60year%60%0A%20%20%20%20BETWEEN%20%222015-11-01T17%3A28%3A08%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20AND%20%222022-03-14T17%3A28%3A08%22%20%3A%3A%20floating_timestamp")
import json
populations = json.loads(response.text)

# 2nd merge: compiling additional data on other district education metrics (from the current year) & joining with the 1st merge
df_ada = pd.read_csv(data_dir + "2021-22 K12 ADA.csv")
df_math = pd.read_csv(data_dir + "2021-22 ISASP MATH.csv").fillna(0)
df_ela = pd.read_csv(data_dir + "2021-22 ISASP ELA.csv").fillna(0)
df_ispp = pd.read_csv(data_dir + "2021-22 ISPP SUMMARY.csv").fillna(0)
df_frpl = pd.read_csv(data_dir + "2021-22 FRPL.csv")
df_iep = pd.read_csv(data_dir + "2021-22 IEP.csv")
df_el = pd.read_csv(data_dir + "2021-22 EL.csv")

In [ ]:
# list of districts that are members of the Urban Education Network of Iowa
urban = ["Ames Comm School District", "Ankeny Comm School District", "Bettendorf Comm School District",
         "Burlington Comm School District", "Cedar Falls Comm School District", "Cedar Rapids Comm School District",
         "College Comm School District", "Council Bluffs Comm School District", "Davenport Comm School District",
         "Des Moines Independent Comm School District", "Dubuque Comm School District", "Fort Dodge Comm School District",
         "Iowa City Comm School District", "Linn-Mar Comm School District", "Marshalltown Comm School District",
         "Mason City Comm School District", "Muscatine Comm School District", "Ottumwa Comm School District",
         "Sioux City Comm School District", "Southeast Polk Comm School District", "Storm Lake Comm School District",
         "Waterloo Comm School District", "West Des Moines Comm School District"]

# **Initial Merge**

In [ ]:
# cleaning the previous, previous year enrollment file
# list of districts that are not community school districts / were incorrectly formatted
need_fix = ["Eddyville-Blakesburg-", "Central DeWitt", "Decorah Community", "Estherville Lincoln",
            "Olin Consolidated", "West Burlington Ind", "South O'Brien", "South Tama County", "Marion Independent",
            "Janesville Consolidated", "Highland", "Exira-Elk Horn-", "Danville", "Colo-NESCO", "Central DeWitt"]

df_ppy_enrollment.columns = df_ppy_enrollment.iloc[3]
df_ppy_enrollment = df_ppy_enrollment.iloc[4:331, :]
df_ppy_enrollment = df_ppy_enrollment.reset_index()
df_ppy_enrollment.columns.name = ''
df_ppy_enrollment.rename(columns = {'Total PK12':'2019-20 Enrollment', "DISTRICT NAME": "District Name"}, inplace = True)

# formatting district names to be compatible with the other datasets
for district in range(len(df_ppy_enrollment)):
  if df_ppy_enrollment.loc[district, "District Name"] not in need_fix:
    df_ppy_enrollment.loc[district, "District Name"] = df_ppy_enrollment.loc[district, "District Name"] + " Comm School District"
df_ppy_enrollment["District Name"] = df_ppy_enrollment['District Name'].replace({"Eddyville-Blakesburg-": "Eddyville-Blakesburg- Fremont CSD",
                                                                               "Decorah Community": "Decorah Community School District",
                                                                               "Estherville Lincoln": "Estherville Lincoln Central Com Sch Dist",
                                                                               "Olin Consolidated": "Olin Consolidated School District",
                                                                               "West Burlington Ind": "West Burlington Ind School District",
                                                                               "South O'Brien": "South O'Brien  Comm School District",
                                                                               "Marion Independent": "Marion Independent School District",
                                                                               "Janesville Consolidated": "Janesville Consolidated School District",
                                                                               "Highland": "Highland  Comm School District",
                                                                               "Exira-Elk Horn-": "Exira-Elk Horn- Kimballton Comm Sch Dist",
                                                                               "Danville": "Danville  Comm School District",
                                                                               "Colo-NESCO": "Colo-NESCO  Comm School District",
                                                                               "Central DeWitt": "Central DeWitt School District"})
df_ppy_enrollment["2019-20 Enrollment"] = df_ppy_enrollment["2019-20 Enrollment"].astype(float)

# extracting the columns of interest
df_ppy_enrollment = df_ppy_enrollment[["District Name", "2019-20 Enrollment"]]
df_ppy_enrollment

,District Name,2019-20 Enrollment
0,AGWSR Comm School District,670.0
1,Adair-Casey Comm School District,259.0
2,Adel DeSoto Minburn Comm School District,2046.0
3,Akron Westfield Comm School District,642.0
4,Albert City-Truesdale Comm School District,114.0
...,...,...
322,Winfield-Mt Union Comm School District,429.0
323,Winterset Comm School District,1795.0
324,Woodbine Comm School District,512.0
325,Woodbury Central Comm School District,620.0


In [ ]:
# cleaning the previous year enrollment file
# list of districts that are not community school districts / were incorrectly formatted
need_fix = ["Eddyville-Blakesburg- Fremont", "Central DeWitt", "Decorah Community", "Estherville Lincoln",
            "Olin Consolidated", "West Burlington Ind", "South O'Brien", "South Tama County", "Marion Independent",
            "Janesville Consolidated", "Highland", "Exira-Elk Horn- Kimballton", "Danville", "Colo-NESCO", "Central DeWitt"]

df_py_enrollment.columns = df_py_enrollment.iloc[3]
df_py_enrollment = df_py_enrollment.iloc[4:331, :]
df_py_enrollment = df_py_enrollment.reset_index()
df_py_enrollment.columns.name = ''
df_py_enrollment.rename(columns = {'Total PK12':'2020-21 Enrollment', "DISTRICT NAME": "District Name"}, inplace = True)

# formatting district names to be compatible with the other datasets
for district in range(len(df_py_enrollment)):
  if df_py_enrollment.loc[district, "District Name"] not in need_fix:
    df_py_enrollment.loc[district, "District Name"] = df_py_enrollment.loc[district, "District Name"] + " Comm School District"
df_py_enrollment["District Name"] = df_py_enrollment['District Name'].replace({"Eddyville-Blakesburg- Fremont": "Eddyville-Blakesburg- Fremont CSD",
                                                                               "Decorah Community": "Decorah Community School District",
                                                                               "Estherville Lincoln": "Estherville Lincoln Central Com Sch Dist",
                                                                               "Olin Consolidated": "Olin Consolidated School District",
                                                                               "West Burlington Ind": "West Burlington Ind School District",
                                                                               "South O'Brien": "South O'Brien  Comm School District",
                                                                               "Marion Independent": "Marion Independent School District",
                                                                               "Janesville Consolidated": "Janesville Consolidated School District",
                                                                               "Highland": "Highland  Comm School District",
                                                                               "Exira-Elk Horn- Kimballton": "Exira-Elk Horn- Kimballton Comm Sch Dist",
                                                                               "Danville": "Danville  Comm School District",
                                                                               "Colo-NESCO": "Colo-NESCO  Comm School District",
                                                                               "Central DeWitt": "Central DeWitt School District"})
df_py_enrollment["2020-21 Enrollment"] = df_py_enrollment["2020-21 Enrollment"].astype(float)

# extracting the columns of interest
df_py_enrollment = df_py_enrollment[["District Name", "2020-21 Enrollment"]]
df_py_enrollment

,District Name,2020-21 Enrollment
0,AGWSR Comm School District,699.0
1,Adair-Casey Comm School District,260.0
2,Adel DeSoto Minburn Comm School District,2110.0
3,Akron Westfield Comm School District,589.0
4,Albert City-Truesdale Comm School District,117.0
...,...,...
322,Winfield-Mt Union Comm School District,412.0
323,Winterset Comm School District,1713.0
324,Woodbine Comm School District,519.0
325,Woodbury Central Comm School District,584.0


In [ ]:
# cleaning the current year enrollment file
df_cy_enrollment.columns = df_cy_enrollment.iloc[3]
df_cy_enrollment = df_cy_enrollment.iloc[4:331, :]
df_cy_enrollment = df_cy_enrollment.reset_index()
df_cy_enrollment.columns.name = ''
df_cy_enrollment.rename(columns = {'COUNTY NAME': "County Name", 'Total PK12':'2021-22 Enrollment', "DISTRICT NAME": "District Name"}, inplace = True)
df_cy_enrollment.loc[:, "PK":] = df_cy_enrollment.loc[:, "PK":].replace(",", "", regex=True).astype(float)

# calculating the racial distribution of a district's enrollment
for race in ["White", "Hispanic", "Asian", "Black", "Multi-Race"]:
  df_cy_enrollment["Percent " + race] = (df_cy_enrollment[race + " Total"] / df_cy_enrollment["2021-22 Enrollment"])

# adding an urban variable based on the list of UENI members
df_cy_enrollment["Urban"] = False
for i in range(len(df_cy_enrollment)):
  if df_cy_enrollment["District Name"][i] in urban:
    df_cy_enrollment.loc[i, "Urban"] = True

# extracting the columns of interest
df_cy_enrollment = df_cy_enrollment[["District Name", "2021-22 Enrollment", "County Name", "Urban",
                                     "Percent White", "Percent Hispanic", "Percent Asian", "Percent Black", "Percent Multi-Race"]]
df_cy_enrollment

,District Name,2021-22 Enrollment,County Name,Urban,Percent White,Percent Hispanic,Percent Asian,Percent Black,Percent Multi-Race
0,AGWSR Comm School District,690.0,Hardin,False,0.910145,0.069565,0.001449,0.002899,0.015942
1,Adair-Casey Comm School District,264.0,Guthrie,False,0.878788,0.094697,0.000000,0.000000,0.015152
2,Adel DeSoto Minburn Comm School District,2149.0,Dallas,False,0.922289,0.038157,0.007911,0.006980,0.022336
3,Akron Westfield Comm School District,613.0,Plymouth,False,0.864600,0.081566,0.003263,0.009788,0.027732
4,Albert City-Truesdale Comm School District,103.0,Buena Vista,False,0.834951,0.135922,0.000000,0.000000,0.029126
...,...,...,...,...,...,...,...,...,...
322,Winfield-Mt Union Comm School District,416.0,Henry,False,0.872596,0.088942,0.000000,0.007212,0.026442
323,Winterset Comm School District,1767.0,Madison,False,0.928127,0.037917,0.006225,0.003396,0.022637
324,Woodbine Comm School District,532.0,Harrison,False,0.954887,0.024436,0.005639,0.000000,0.011278
325,Woodbury Central Comm School District,587.0,Woodbury,False,0.952300,0.027257,0.000000,0.000000,0.020443


In [ ]:
# cleaning the next year enrollment file
df_ny_enrollment.columns = df_ny_enrollment.iloc[3]
df_ny_enrollment = df_ny_enrollment.iloc[4:331, :]
df_ny_enrollment = df_ny_enrollment.reset_index()
df_ny_enrollment.columns.name = ''
df_ny_enrollment.rename(columns = {'Total PK12':'2022-23 Enrollment', "DISTRICT NAME": "District Name"}, inplace = True)
df_ny_enrollment["2022-23 Enrollment"] = df_ny_enrollment["2022-23 Enrollment"].astype(float)

# extracting the columns of interest
df_ny_enrollment = df_ny_enrollment[["District Name", "2022-23 Enrollment"]]
df_ny_enrollment

,District Name,2022-23 Enrollment
0,AGWSR Comm School District,650.0
1,Adair-Casey Comm School District,271.0
2,Adel DeSoto Minburn Comm School District,2254.0
3,Akron Westfield Comm School District,608.0
4,Albert City-Truesdale Comm School District,114.0
...,...,...
322,Winfield-Mt Union Comm School District,391.0
323,Winterset Comm School District,1683.0
324,Woodbine Comm School District,534.0
325,Woodbury Central Comm School District,566.0


In [ ]:
# cleaning the state's enrollment projections file & extracting their projections for 2022-23
df_enrollment_proj.columns = df_enrollment_proj.iloc[2]
df_enrollment_proj = df_enrollment_proj.iloc[3:330, :]
df_enrollment_proj = df_enrollment_proj.reset_index()
df_enrollment_proj.columns.name = ''
df_enrollment_proj.rename(columns = {'District Name ': 'District Name', '2022-23': "2022-23 State Enrollment Projection"}, inplace=True)
df_enrollment_proj["2022-23 State Enrollment Projection"] = df_enrollment_proj["2022-23 State Enrollment Projection"].replace(",", "", regex=True).astype(float)

# extracting the columns of interest
df_enrollment_proj = df_enrollment_proj[["District Name", "2022-23 State Enrollment Projection"]]
df_enrollment_proj

,District Name,2022-23 State Enrollment Projection
0,AGWSR Comm School District,702.0
1,Adair-Casey Comm School District,290.0
2,Adel DeSoto Minburn Comm School District,2163.0
3,Akron Westfield Comm School District,556.0
4,Albert City-Truesdale Comm School District,194.0
...,...,...
322,Winfield-Mt Union Comm School District,314.0
323,Winterset Comm School District,1667.0
324,Woodbine Comm School District,474.0
325,Woodbury Central Comm School District,508.0


In [ ]:
# merging all of the cleaned enrollment files together
df_merged = df_ppy_enrollment.merge(df_py_enrollment.merge(df_cy_enrollment.merge(df_ny_enrollment.merge(df_enrollment_proj))), on="District Name", how="outer")
df_merged

,District Name,2019-20 Enrollment,2020-21 Enrollment,2021-22 Enrollment,County Name,Urban,Percent White,Percent Hispanic,Percent Asian,Percent Black,Percent Multi-Race,2022-23 Enrollment,2022-23 State Enrollment Projection
0,AGWSR Comm School District,670.0,699.0,690.0,Hardin,False,0.910145,0.069565,0.001449,0.002899,0.015942,650.0,702.0
1,Adair-Casey Comm School District,259.0,260.0,264.0,Guthrie,False,0.878788,0.094697,0.000000,0.000000,0.015152,271.0,290.0
2,Adel DeSoto Minburn Comm School District,2046.0,2110.0,2149.0,Dallas,False,0.922289,0.038157,0.007911,0.006980,0.022336,2254.0,2163.0
3,Akron Westfield Comm School District,642.0,589.0,613.0,Plymouth,False,0.864600,0.081566,0.003263,0.009788,0.027732,608.0,556.0
4,Albert City-Truesdale Comm School District,114.0,117.0,103.0,Buena Vista,False,0.834951,0.135922,0.000000,0.000000,0.029126,114.0,194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Winfield-Mt Union Comm School District,429.0,412.0,416.0,Henry,False,0.872596,0.088942,0.000000,0.007212,0.026442,391.0,314.0
323,Winterset Comm School District,1795.0,1713.0,1767.0,Madison,False,0.928127,0.037917,0.006225,0.003396,0.022637,1683.0,1667.0
324,Woodbine Comm School District,512.0,519.0,532.0,Harrison,False,0.954887,0.024436,0.005639,0.000000,0.011278,534.0,474.0
325,Woodbury Central Comm School District,620.0,584.0,587.0,Woodbury,False,0.952300,0.027257,0.000000,0.000000,0.020443,566.0,508.0


In [ ]:
# calculating the MSE of the state's 2022-23 enrollment projections w/ actual enrollment figures
from sklearn.metrics import mean_squared_error
mean_squared_error(df_merged["2022-23 Enrollment"], df_merged["2022-23 State Enrollment Projection"]) ** .5
# were off by an average of 177 students!

177.25901503107235

In [ ]:
# normalizing & cleaning the populations json file
df_populations = pd.json_normalize(populations)
df_populations.rename(columns={"population": "Population", "geographicname": "County Name", ":@computed_region_hhz5_dst4": "Rating Area", "year": "Year"}, inplace=True)
df_populations["County Name"] = df_populations["County Name"].replace(" County", "", regex=True)
for i in range(len(df_populations["Year"])):
  df_populations["Year"][i] = df_populations["Year"][i][:4]

# extracting 2019, 2020, & 2021 population data by county
df_populations2021 = df_populations[df_populations["Year"] == "2021"]
df_populations2020 = df_populations[df_populations["Year"] == "2020"]
df_populations2019 = df_populations[df_populations["Year"] == "2019"]

# merging these dfs together and then calculating the change from 2019-21 & 2020-21
df_populations_merged = df_populations2019.merge(df_populations2020, on="County Name", suffixes=(" 2019", " 2020")).merge(df_populations2021, on="County Name")
df_populations_merged.rename(columns={"Population":"Population 2021"}, inplace=True)
df_populations_merged["Previous 2 Year Population Change"] = (df_populations_merged["Population 2021"].astype(float) - df_populations_merged["Population 2019"].astype(float)) / df_populations_merged["Population 2019"].astype(float)
df_populations_merged["Previous Year Population Change"] = (df_populations_merged["Population 2021"].astype(float) - df_populations_merged["Population 2020"].astype(float)) / df_populations_merged["Population 2020"].astype(float)

# extracting the columns of interest
df_populations_merged = df_populations_merged[["County Name", "Rating Area", "Previous 2 Year Population Change", "Previous Year Population Change"]]
df_populations_merged

,County Name,Rating Area,Previous 2 Year Population Change,Previous Year Population Change
0,Chickasaw,7,-0.003855,-0.010406
1,Benton,6,0.001794,0.004536
2,Jefferson,5,-0.144739,-0.001022
3,Muscatine,5,0.000563,-0.012652
4,Grundy,1,0.009402,0.001460
...,...,...,...,...
94,Winneshiek,7,-0.004952,-0.008869
95,Delaware,6,0.029334,0.001258
96,Bremer,7,0.000758,0.003722
97,Buena Vista,3,0.058665,-0.002497


In [ ]:
# merging the populations df with the merged enrollment df
df_merged1 = df_merged.merge(df_populations_merged, on="County Name", how="outer")
df_merged1["Previous Year Enrollment Change"] = (df_merged1["2021-22 Enrollment"] - df_merged1["2020-21 Enrollment"]) / df_merged1["2020-21 Enrollment"]
df_merged1["Previous 2 Year Enrollment Change"] = (df_merged1["2021-22 Enrollment"] - df_merged1["2019-20 Enrollment"].astype(float)) / df_merged1["2019-20 Enrollment"].astype(float)
df_merged1["Next Year Enrollment Change"]= (df_merged1["2022-23 Enrollment"] - df_merged1["2021-22 Enrollment"]) / df_merged1["2021-22 Enrollment"]
df_merged1

,District Name,2019-20 Enrollment,2020-21 Enrollment,2021-22 Enrollment,County Name,Urban,Percent White,Percent Hispanic,Percent Asian,Percent Black,Percent Multi-Race,2022-23 Enrollment,2022-23 State Enrollment Projection,Rating Area,Previous 2 Year Population Change,Previous Year Population Change,Previous Year Enrollment Change,Previous 2 Year Enrollment Change,Next Year Enrollment Change
0,AGWSR Comm School District,670.0,699.0,690.0,Hardin,False,0.910145,0.069565,0.001449,0.002899,0.015942,650.0,702.0,1,-0.008192,-0.010072,-0.012876,0.029851,-0.057971
1,Alden Comm School District,173.0,169.0,179.0,Hardin,False,0.849162,0.094972,0.000000,0.016760,0.039106,159.0,279.0,1,-0.008192,-0.010072,0.059172,0.034682,-0.111732
2,Eldora-New Providence Comm School District,574.0,529.0,518.0,Hardin,False,0.882239,0.081081,0.005792,0.007722,0.017375,540.0,544.0,1,-0.008192,-0.010072,-0.020794,-0.097561,0.042471
3,Hubbard-Radcliffe Comm School District,398.0,413.0,375.0,Hardin,False,0.882667,0.061333,0.005333,0.010667,0.037333,377.0,447.0,1,-0.008192,-0.010072,-0.092010,-0.057789,0.005333
4,Iowa Falls Comm School District,1314.0,1249.0,1275.0,Hardin,False,0.868235,0.084706,0.003922,0.010980,0.031373,1278.0,984.0,1,-0.008192,-0.010072,0.020817,-0.029680,0.002353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Sibley-Ocheyedan Comm School District,769.0,766.0,730.0,Osceola,False,0.739726,0.205479,0.004110,0.002740,0.015068,729.0,716.0,3,0.033736,-0.005329,-0.046997,-0.050715,-0.001370
323,South Hamilton Comm School District,721.0,688.0,699.0,Hamilton,False,0.862661,0.098712,0.005722,0.004292,0.027182,690.0,627.0,1,0.007717,-0.010107,0.015988,-0.030513,-0.012876
324,Stratford Comm School District,65.0,64.0,66.0,Hamilton,False,0.954545,0.030303,0.000000,0.000000,0.000000,80.0,126.0,1,0.007717,-0.010107,0.031250,0.015385,0.212121
325,Webster City Comm School District,1904.0,1855.0,1887.0,Hamilton,False,0.686804,0.225225,0.047695,0.004240,0.033386,1827.0,1762.0,1,0.007717,-0.010107,0.017251,-0.008929,-0.031797


# **Second Merge**

In [ ]:
# cleaning the K12 ADA (average daily attendance) file
df_ada.columns = df_ada.iloc[4]
df_ada = df_ada.iloc[6:, :]
df_ada = df_ada.reset_index()
df_ada.columns.name = ''
df_ada.rename(columns = {'ADA_%_K12':'Percent ADA'}, inplace=True)

# extracting the columns of interest
df_ada = df_ada[["District Name", "Percent ADA"]]
df_ada

,District Name,Percent ADA
0,AGWSR Comm School District,92.1
1,Adair-Casey Comm School District,94.5
2,Adel DeSoto Minburn Comm School District,94.1
3,Akron Westfield Comm School District,94
4,Albert City-Truesdale Comm School District,92.2
...,...,...
322,Winfield-Mt Union Comm School District,92
323,Winterset Comm School District,93.7
324,Woodbine Comm School District,94.4
325,Woodbury Central Comm School District,93.8


In [ ]:
# cleaning the standardized math testing file
df_math.columns = df_math.iloc[5]
df_math.columns.name = ''
df_math = df_math.iloc[6:333, :]
df_math = df_math.drop(columns=[0.0])
df_math = df_math.replace("small N", "1")
df_math.loc[:, "Grade 3 Proficient":] = df_math.loc[:, "Grade 3 Proficient":].replace(",", "", regex=True).astype(float)
df_math

,County,County Name,AEA,District #,District Name,Grade 3 Not Proficient,Grade 3 Proficient,Grade 3 Total Tested,Grade 3 % Proficient,Grade 4 Not Proficient,...,Grade 9 Total Tested,Grade 9 % Proficient,Grade 10 Not Proficient,Grade 10 Proficient,Grade 10 Total Tested,Grade 10 % Proficient,Grade 11 Not Proficient,Grade 11 Proficient,Grade 11 Total Tested,Grade 11 % Proficient
6,42,Hardin,07,0009,AGWSR Comm School District,13,31.0,44.0,70.5,11.0,...,56.0,53.6,16.0,24.0,40.0,60.0,16.0,28.0,44.0,63.6
7,39,Guthrie,11,0018,Adair-Casey Comm School District,10,12.0,22.0,54.5,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,25,Dallas,11,0027,Adel DeSoto Minburn Comm School District,21,133.0,154.0,86.4,22.0,...,168.0,83.3,33.0,123.0,156.0,78.8,22.0,106.0,128.0,82.8
9,75,Plymouth,12,0063,Akron Westfield Comm School District,3,39.0,42.0,92.9,2.0,...,57.0,71.9,8.0,37.0,45.0,82.2,17.0,27.0,44.0,61.4
10,11,Buena Vista,05,0072,Albert City-Truesdale Comm School District,1,16.0,17.0,94.1,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,44,Henry,15,7047,Winfield-Mt Union Comm School District,7,19.0,26.0,73.1,9.0,...,24.0,33.3,18.0,15.0,33.0,45.5,15.0,21.0,36.0,58.3
329,61,Madison,11,7056,Winterset Comm School District,34,71.0,105.0,67.6,38.0,...,131.0,79.4,55.0,85.0,140.0,60.7,48.0,84.0,132.0,63.6
330,43,Harrison,13,7092,Woodbine Comm School District,8,19.0,27.0,70.4,8.0,...,33.0,63.6,10.0,20.0,30.0,66.7,11.0,16.0,27.0,59.3
331,97,Woodbury,12,7098,Woodbury Central Comm School District,6,31.0,37.0,83.8,2.0,...,45.0,57.8,21.0,25.0,46.0,54.3,20.0,18.0,38.0,47.4


In [ ]:
# cleaning the standardized ELA testing file
df_ela.columns = df_ela.iloc[5]
df_ela.columns.name = ''
df_ela = df_ela.iloc[6:333, :]
df_ela = df_ela.drop(columns=[0.0])
df_ela = df_ela.replace("small N", "1")
df_ela.loc[:, "Grade 3 Proficient":] = df_ela.loc[:, "Grade 3 Proficient":].replace(",", "", regex=True).astype(float)
df_ela

,County,County Name,AEA,District #,District Name,Grade 3 Not Proficient,Grade 3 Proficient,Grade 3 Total Tested,Grade 3 % Proficient,Grade 4 Not Proficient,...,Grade 9 Total Tested,Grade 9 % Proficient,Grade 10 Not Proficient,Grade 10 Proficient,Grade 10 Total Tested,Grade 10 % Proficient,Grade 11 Not Proficient,Grade 11 Proficient,Grade 11 Total Tested,Grade 11 % Proficient
6,42,Hardin,07,0009,AGWSR Comm School District,22,22.0,44.0,50.0,11.0,...,56.0,69.6,9.0,31.0,40.0,77.5,9.0,35.0,44.0,79.5
7,39,Guthrie,11,0018,Adair-Casey Comm School District,10,12.0,22.0,54.5,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,25,Dallas,11,0027,Adel DeSoto Minburn Comm School District,29,125.0,154.0,81.2,18.0,...,168.0,86.9,16.0,140.0,156.0,89.7,18.0,110.0,128.0,85.9
9,75,Plymouth,12,0063,Akron Westfield Comm School District,6,36.0,42.0,85.7,2.0,...,57.0,73.7,9.0,36.0,45.0,80.0,15.0,29.0,44.0,65.9
10,11,Buena Vista,05,0072,Albert City-Truesdale Comm School District,2,15.0,17.0,88.2,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,44,Henry,15,7047,Winfield-Mt Union Comm School District,8,18.0,26.0,69.2,2.0,...,24.0,50.0,14.0,19.0,33.0,57.6,7.0,29.0,36.0,80.6
329,61,Madison,11,7056,Winterset Comm School District,38,69.0,107.0,64.5,33.0,...,131.0,80.9,42.0,98.0,140.0,70.0,45.0,86.0,131.0,65.6
330,43,Harrison,13,7092,Woodbine Comm School District,10,17.0,27.0,63.0,5.0,...,33.0,84.8,6.0,24.0,30.0,80.0,6.0,19.0,25.0,76.0
331,97,Woodbury,12,7098,Woodbury Central Comm School District,8,29.0,37.0,78.4,3.0,...,45.0,93.3,8.0,38.0,46.0,82.6,6.0,32.0,38.0,84.2


In [ ]:
# calculating proficiency rates within the math & ELA dfs
proficient = []
total = []
for i in range(3, 12):
  proficient.append("Grade " + str(i) + " Proficient")
  total.append("Grade " + str(i) + " Total Tested")
df_math["Math Proficiency Rate"] = (df_math[proficient].sum(axis=1)) / (df_math[total].sum(axis=1))
df_ela["ELA Proficiency Rate"] = (df_ela[proficient].sum(axis=1)) / (df_ela[total].sum(axis=1))

# extracting the columns of interest
df_math = df_math[["District Name", "Math Proficiency Rate"]]
df_ela = df_ela[["District Name", "ELA Proficiency Rate"]]

In [ ]:
df_math

,District Name,Math Proficiency Rate
6,AGWSR Comm School District,0.573171
7,Adair-Casey Comm School District,0.635870
8,Adel DeSoto Minburn Comm School District,0.802768
9,Akron Westfield Comm School District,0.809406
10,Albert City-Truesdale Comm School District,0.764706
...,...,...
328,Winfield-Mt Union Comm School District,0.533865
329,Winterset Comm School District,0.699643
330,Woodbine Comm School District,0.638225
331,Woodbury Central Comm School District,0.694915


In [ ]:
df_ela

,District Name,ELA Proficiency Rate
6,AGWSR Comm School District,0.678049
7,Adair-Casey Comm School District,0.639344
8,Adel DeSoto Minburn Comm School District,0.853979
9,Akron Westfield Comm School District,0.823821
10,Albert City-Truesdale Comm School District,0.745098
...,...,...
328,Winfield-Mt Union Comm School District,0.641434
329,Winterset Comm School District,0.717217
330,Woodbine Comm School District,0.773196
331,Woodbury Central Comm School District,0.833333


In [ ]:
# cleaning the ispp (yearly state district performance assessment) file
df_ispp.rename(columns = {'District_Name':'District Name', "School_Index": "ISPP Index"}, inplace=True)
df_ispp_indexes = df_ispp.groupby(by="District Name")[["ISPP Index"]].mean()
df_ispp_indexes = df_ispp_indexes.reset_index()
df_ispp_indexes["District Name"] = df_ispp_indexes['District Name'].replace({'South Tama County ': "South Tama County"})

# extracting the columns of interest
df_ispp_indexes = df_ispp_indexes[["District Name", "ISPP Index"]]
df_ispp_indexes

,District Name,ISPP Index
0,AGWSR Comm School District,51.372500
1,AHSTW Comm School District,52.846667
2,Adair-Casey Comm School District,51.725000
3,Adel DeSoto Minburn Comm School District,64.002000
4,Akron Westfield Comm School District,56.443333
...,...,...
322,Winfield-Mt Union Comm School District,55.845000
323,Winterset Comm School District,54.327500
324,Woodbine Comm School District,54.180000
325,Woodbury Central Comm School District,57.573333


In [ ]:
# cleaning the frpl (free/reduced price lunch) file
df_frpl.columns = df_frpl.iloc[5]
df_frpl = df_frpl.iloc[6:333]
df_frpl = df_frpl.reset_index()
df_frpl.columns.name = ''
df_frpl.rename(columns = {"District_Name":"District Name"}, inplace=True)
df_frpl["Percent Free/Reduced Price Lunch"] = df_frpl["Percent Free/Reduced Price Lunch"].astype(float) / 100

# extracting the columns of interest
df_frpl = df_frpl[["District Name", "Percent Free/Reduced Price Lunch"]]
df_frpl

,District Name,Percent Free/Reduced Price Lunch
0,AGWSR Comm School District,0.4311
1,Adair-Casey Comm School District,0.4120
2,Adel DeSoto Minburn Comm School District,0.1855
3,Akron Westfield Comm School District,0.2997
4,Albert City-Truesdale Comm School District,0.5217
...,...,...
322,Winfield-Mt Union Comm School District,0.5271
323,Winterset Comm School District,0.3180
324,Woodbine Comm School District,0.3674
325,Woodbury Central Comm School District,0.2868


In [ ]:
# cleaning the IEP (individualized education program) file
df_iep.columns = df_iep.iloc[2]
df_iep.rename(columns = {'DISTRICT NAME':'District Name'}, inplace=True)
df_iep = df_iep.iloc[3:330]
df_iep = df_iep.reset_index()
df_iep.columns.name = ''
df_iep["Percent IEP"] = (df_iep["All IEP Students"].replace(",", "", regex=True).astype(float) / df_iep["All Students"].replace(",", "", regex=True).astype(float))

# extracting the columns of interest
df_iep = df_iep[["District Name", "Percent IEP"]]
df_iep

,District Name,Percent IEP
0,AGWSR Comm School District,0.142029
1,Adair-Casey Comm School District,0.132576
2,Adel DeSoto Minburn Comm School District,0.097720
3,Akron Westfield Comm School District,0.125612
4,Albert City-Truesdale Comm School District,0.223301
...,...,...
322,Winfield-Mt Union Comm School District,0.112981
323,Winterset Comm School District,0.124505
324,Woodbine Comm School District,0.125940
325,Woodbury Central Comm School District,0.122658


In [ ]:
# cleaning the EL (English-learners) file
df_el.columns = df_el.iloc[4]
df_el = df_el.iloc[5:332]
df_el = df_el.reset_index()
df_el.columns.name = ''
df_el["Percent EL"] = df_el["Percent EL"].replace("%", "", regex=True).astype(float) / 100

# extracting the columns of interest
df_el = df_el[["District Name", "Percent EL"]]
df_el

,District Name,Percent EL
0,AGWSR Comm School District,0.011
1,Adair-Casey Comm School District,0.024
2,Adel DeSoto Minburn Comm School District,0.009
3,Akron Westfield Comm School District,0.002
4,Albert City-Truesdale Comm School District,0.033
...,...,...
322,Winfield-Mt Union Comm School District,0.016
323,Winterset Comm School District,0.011
324,Woodbine Comm School District,0.000
325,Woodbury Central Comm School District,0.000


In [ ]:
# merging all of the 2021-22 school year metrics with the previous merge
df_districts = df_merged1.merge(df_ada.merge(df_math.merge(df_ela.merge(df_ispp_indexes.merge(df_frpl.merge(df_iep.merge(df_el)))))), on="District Name", how="outer")
df_districts

,District Name,2019-20 Enrollment,2020-21 Enrollment,2021-22 Enrollment,County Name,Urban,Percent White,Percent Hispanic,Percent Asian,Percent Black,...,Previous Year Enrollment Change,Previous 2 Year Enrollment Change,Next Year Enrollment Change,Percent ADA,Math Proficiency Rate,ELA Proficiency Rate,ISPP Index,Percent Free/Reduced Price Lunch,Percent IEP,Percent EL
0,AGWSR Comm School District,670.0,699.0,690.0,Hardin,False,0.910145,0.069565,0.001449,0.002899,...,-0.012876,0.029851,-0.057971,92.1,0.573171,0.678049,51.372500,0.4311,0.142029,0.011
1,Alden Comm School District,173.0,169.0,179.0,Hardin,False,0.849162,0.094972,0.000000,0.016760,...,0.059172,0.034682,-0.111732,91.3,0.607595,0.670886,56.190000,0.4056,0.128492,0.049
2,Eldora-New Providence Comm School District,574.0,529.0,518.0,Hardin,False,0.882239,0.081081,0.005792,0.007722,...,-0.020794,-0.097561,0.042471,93,0.650519,0.697917,53.070000,0.4235,0.173745,0.017
3,Hubbard-Radcliffe Comm School District,398.0,413.0,375.0,Hardin,False,0.882667,0.061333,0.005333,0.010667,...,-0.092010,-0.057789,0.005333,92.8,0.736434,0.697674,53.635000,0.3983,0.133333,0.011
4,Iowa Falls Comm School District,1314.0,1249.0,1275.0,Hardin,False,0.868235,0.084706,0.003922,0.010980,...,0.020817,-0.029680,0.002353,93.5,0.607692,0.762516,57.357500,0.3898,0.131765,0.031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Sibley-Ocheyedan Comm School District,769.0,766.0,730.0,Osceola,False,0.739726,0.205479,0.004110,0.002740,...,-0.046997,-0.050715,-0.001370,94.5,0.703297,0.746137,51.450000,0.5130,0.126027,0.103
323,South Hamilton Comm School District,721.0,688.0,699.0,Hamilton,False,0.862661,0.098712,0.005722,0.004292,...,0.015988,-0.030513,-0.012876,93.2,0.802784,0.849188,61.995000,0.2395,0.092990,0.021
324,Stratford Comm School District,65.0,64.0,66.0,Hamilton,False,0.954545,0.030303,0.000000,0.000000,...,0.031250,0.015385,0.212121,92.9,0.846154,0.846154,54.440000,0.4528,0.106061,0.000
325,Webster City Comm School District,1904.0,1855.0,1887.0,Hamilton,False,0.686804,0.225225,0.047695,0.004240,...,0.017251,-0.008929,-0.031797,90.7,0.651613,0.657143,55.118000,0.3769,0.104928,0.147


In [ ]:
# rearranging columns
df_districts = df_districts[["District Name", "County Name", "Rating Area",
                             "2019-20 Enrollment", "2020-21 Enrollment", "Previous 2 Year Enrollment Change", "Previous Year Enrollment Change",
                             "Previous 2 Year Population Change", "Previous Year Population Change", "2021-22 Enrollment",
                             "Urban", "Percent White", "Percent Hispanic", "Percent Asian", "Percent Black", "Percent Multi-Race",
                             "Percent ADA", "Math Proficiency Rate", "ELA Proficiency Rate", "ISPP Index", "Percent Free/Reduced Price Lunch", "Percent IEP", "Percent EL",
                             "2022-23 State Enrollment Projection", "2022-23 Enrollment", "Next Year Enrollment Change"]]
df_districts

,District Name,County Name,Rating Area,2019-20 Enrollment,2020-21 Enrollment,Previous 2 Year Enrollment Change,Previous Year Enrollment Change,Previous 2 Year Population Change,Previous Year Population Change,2021-22 Enrollment,...,Percent ADA,Math Proficiency Rate,ELA Proficiency Rate,ISPP Index,Percent Free/Reduced Price Lunch,Percent IEP,Percent EL,2022-23 State Enrollment Projection,2022-23 Enrollment,Next Year Enrollment Change
0,AGWSR Comm School District,Hardin,1,670.0,699.0,0.029851,-0.012876,-0.008192,-0.010072,690.0,...,92.1,0.573171,0.678049,51.372500,0.4311,0.142029,0.011,702.0,650.0,-0.057971
1,Alden Comm School District,Hardin,1,173.0,169.0,0.034682,0.059172,-0.008192,-0.010072,179.0,...,91.3,0.607595,0.670886,56.190000,0.4056,0.128492,0.049,279.0,159.0,-0.111732
2,Eldora-New Providence Comm School District,Hardin,1,574.0,529.0,-0.097561,-0.020794,-0.008192,-0.010072,518.0,...,93,0.650519,0.697917,53.070000,0.4235,0.173745,0.017,544.0,540.0,0.042471
3,Hubbard-Radcliffe Comm School District,Hardin,1,398.0,413.0,-0.057789,-0.092010,-0.008192,-0.010072,375.0,...,92.8,0.736434,0.697674,53.635000,0.3983,0.133333,0.011,447.0,377.0,0.005333
4,Iowa Falls Comm School District,Hardin,1,1314.0,1249.0,-0.029680,0.020817,-0.008192,-0.010072,1275.0,...,93.5,0.607692,0.762516,57.357500,0.3898,0.131765,0.031,984.0,1278.0,0.002353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Sibley-Ocheyedan Comm School District,Osceola,3,769.0,766.0,-0.050715,-0.046997,0.033736,-0.005329,730.0,...,94.5,0.703297,0.746137,51.450000,0.5130,0.126027,0.103,716.0,729.0,-0.001370
323,South Hamilton Comm School District,Hamilton,1,721.0,688.0,-0.030513,0.015988,0.007717,-0.010107,699.0,...,93.2,0.802784,0.849188,61.995000,0.2395,0.092990,0.021,627.0,690.0,-0.012876
324,Stratford Comm School District,Hamilton,1,65.0,64.0,0.015385,0.031250,0.007717,-0.010107,66.0,...,92.9,0.846154,0.846154,54.440000,0.4528,0.106061,0.000,126.0,80.0,0.212121
325,Webster City Comm School District,Hamilton,1,1904.0,1855.0,-0.008929,0.017251,0.007717,-0.010107,1887.0,...,90.7,0.651613,0.657143,55.118000,0.3769,0.104928,0.147,1762.0,1827.0,-0.031797


In [ ]:
df_districts.to_csv(path_or_buf=data_dir + "df_districts.csv", index=False)